# Import packages

In [ ]:
!pip install -q efficientnet
from keras import backend as K
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, Callback
from keras.layers import Activation, Conv1D, Conv2D, Dense, Dropout, Flatten, Input, average, concatenate, MaxPool1D, MaxPooling2D, MaxPool2D
from keras.models import Sequential, Model, load_model
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import efficientnet.tfkeras as efn

In [ ]:
trainTab = pd.read_csv("../input/jpeg-melanoma-256x256/train.csv")
testTab = pd.read_csv("../input/jpeg-melanoma-256x256/test.csv")

dim=256
bs=64

target=np.array(trainTab['target'].values)

mali = target == 1
imali = np.arange(0, trainTab.shape[0], 1, dtype='int')
imali = imali[mali]

beni = target == 0
ibeni = np.arange(0, trainTab.shape[0], 1, dtype='int')
ibeni = ibeni[beni]

Rows = np.arange(0, trainTab.shape[0], 1)
trbeni = np.random.choice(ibeni, size=int(len(ibeni)*0.6), replace=False)
vlbeni = np.array([a for a in Rows if not a in trbeni], dtype='int')

trmali=np.repeat(imali, 5)
vlmali=imali

trainRows = np.random.choice([*trbeni, *trmali], size=len(trbeni)+len(trmali), replace=False)
valRows = np.random.choice([*vlmali, *vlbeni], size=len(vlbeni)+len(vlmali), replace=False)

# Fill list with image names jpg and png files

In [ ]:
TrainJPG=[]
for i in range(trainTab.shape[0]):
    TrainJPG.append('../input/jpeg-melanoma-256x256/train/' + str(trainTab['image_name'][i]) + '.jpg')
    
TestJPG=[]
for i in range(testTab.shape[0]):
    TestJPG.append('../input/jpeg-melanoma-256x256/test/' + str(testTab['image_name'][i]) + '.jpg')
    
TrainPNG=[]
for i in range(trainTab.shape[0]):
    TrainPNG.append('../input/landscape/tabFolders256/trainTab/' + str(trainTab['image_name'][i]) + '.png')

TestPNG=[]
for i in range(testTab.shape[0]):
    TestPNG.append('../input/landscape/tabFolders256/testTab/' + str(testTab['image_name'][i]) + '.png')

# Define the image generators

In [ ]:
train_args = dict(brightness_range = [0.5,1.5], rescale=1./255, width_shift_range=0.5, height_shift_range=0.5,
                  shear_range=0.6, zoom_range=0.6, horizontal_flip = True, rotation_range = 90)

val_args = dict(rescale=1./255)

datagen = ImageDataGenerator(**train_args)

Vdatagen = ImageDataGenerator(**val_args)

# Define image indices and batch iterator

In [ ]:
trIndi=np.arange(0, len(trainRows), 1, dtype='int')
vlIndi=np.arange(0, len(valRows), 1, dtype='int')
teIndi=np.arange(0, testTab.shape[0], 1, dtype='int')

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

ggg=np.arange(0,200,1,dtype='int') 

for x in batch(ggg, 125):
    print(len(x))


# Functions to yield images and target to the CNN

In [ ]:
def gT():
    while True:
        for x in batch(vlIndi, bs):
            imArrayJPG=np.zeros((len(x), dim, dim, 3))
            imArrayPNG=np.zeros((len(x), dim, dim, 1))
            for j in range(len(x)):
                ima=load_img(TrainPNG[trainRows[x[j]]], color_mode="grayscale")
                ima=img_to_array(ima)
                imArrayPNG[j,:,:,0]=ima[:,:,0]
                ima=load_img(TrainJPG[trainRows[x[j]]], color_mode="rgb")
                ima=img_to_array(ima)
                imArrayJPG[j,:,:,0:3]=ima[:,:,0:3]
            outPNG=Vdatagen.flow(imArrayPNG, batch_size=len(x), shuffle=False)
            outJPG=datagen.flow(imArrayJPG, batch_size=len(x), shuffle=False)
            salPNG=outPNG.next()
            salJPG=outJPG.next()
            yield [salPNG, salJPG], target[trainRows[x]]

def gV():
    while True:
        for x in batch(vlIndi, bs):
            imArrayJPG=np.zeros((len(x), dim, dim, 3))
            imArrayPNG=np.zeros((len(x), dim, dim, 1))
            for j in range(len(x)):
                ima=load_img(TrainPNG[valRows[x[j]]], color_mode="grayscale")
                ima=img_to_array(ima)
                imArrayPNG[j,:,:,0]=ima[:,:,0]
                ima=load_img(TrainJPG[valRows[x[j]]], color_mode="rgb")
                ima=img_to_array(ima)
                imArrayJPG[j,:,:,0:3]=ima[:,:,0:3]
            outPNG=Vdatagen.flow(imArrayPNG, batch_size=len(x), shuffle=False)
            outJPG=Vdatagen.flow(imArrayJPG, batch_size=len(x), shuffle=False)
            salPNG=outPNG.next()
            salJPG=outJPG.next()
            yield [salPNG, salJPG], target[valRows[x]]

def gTe():
    while True:
        for x in batch(teIndi, 34):
            imArrayJPG=np.zeros((len(x), dim, dim, 3))
            imArrayPNG=np.zeros((len(x), dim, dim, 1))
            for j in range(len(x)):
                ima=load_img(TestPNG[x[j]], color_mode="grayscale")
                ima=img_to_array(ima)
                imArrayPNG[j,:,:,0]=ima[:,:,0]
                ima=load_img(TestJPG[x[j]], color_mode="rgb")
                ima=img_to_array(ima)
                imArrayJPG[j,:,:,0:3]=ima[:,:,0:3]
            outPNG=Vdatagen.flow(imArrayPNG, batch_size=len(x), shuffle=False)
            outJPG=Vdatagen.flow(imArrayJPG, batch_size=len(x), shuffle=False)
            salPNG=outPNG.next()
            salJPG=outJPG.next()
            yield [salPNG, salJPG]

In [ ]:
xu=gT()

a=0
for i in xu:
    if a >= 4:
        break
    a=a+1
    print(i[0][0].shape)
    print(i[0][1].shape)

In [ ]:
plt.figure(figsize=(16, 4.5))

a=0
for i in xu:
    if a >= 20:
        break
    plt.subplot(2, 10, a+1)
    a=a+1
    plt.imshow(i[0][0][a,:,:,0],cmap=plt.cm.binary)
    plt.axis('off')
plt.subplots_adjust(wspace=0.1, hspace=-0.3)
plt.show()

plt.figure(figsize=(16, 4.5))

a=0
for i in xu:
    if a >= 20:
        break
    plt.subplot(2, 10, a+1)
    a=a+1
    plt.imshow(i[0][1][a,:,:,0:3],cmap=plt.cm.binary)
    plt.axis('off')
plt.subplots_adjust(wspace=0.1, hspace=-0.3)
plt.show()

# Define CNNs pretrained and custom models

In [ ]:
ks= 3
x = Sequential()
x.add(Conv2D(24, kernel_size = ks, activation='relu', padding='same', input_shape = (dim, dim, 1)))
x.add(MaxPool2D())
x.add(Conv2D(48, kernel_size = ks, activation='relu', padding='same'))
x.add(MaxPool2D(padding='same'))
x.add(Conv2D(64, kernel_size = ks, activation='relu', padding='same'))
x.add(Flatten())
x.add(Dropout(0.3))
x.add(Dense(128, activation="relu"))

base_model = efn.EfficientNetB0(input_shape=(dim, dim,3), weights='imagenet', include_top=False, pooling='avg')
base_model.trainable = False
y = base_model.output
y = Dense(512)(y)
y = Dropout(0.4)(y)
y = Dense(256)(y)
y = Dropout(0.3)(y)
y = Dense(128)(y)
y = Dense(256, activation='relu')(y)
y = Model(inputs=base_model.input, outputs=y)

combined = concatenate([x.output, y.output], axis=1)
combined= Dense(1, activation="sigmoid")(combined)

model = Model(inputs=[x.input, y.input], outputs=combined)

In [ ]:
from keras import backend as K

def binary_focal_loss(gamma=2., alpha=.75):
    def binary_focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

# COMPILE WITH ADAM OPTIMIZER AND CROSS ENTROPY COST
model.compile(optimizer= 'adam', loss=binary_focal_loss(), metrics=[tf.keras.metrics.AUC()])

filepath='modelEffNet2CNNs.h5'

In [ ]:
# CUSTOM LEARNING SCHEUDLE
LR_START = 0.00001
LR_MAX = 0.001
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 0
LR_STEP_DECAY = 0.75

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = LR_MAX * LR_STEP_DECAY**((epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS)//10)
    return lr


lr2 = LearningRateScheduler(lrfn, verbose = True)

EarLY=EarlyStopping(monitor='val_loss', mode='min', min_delta=0, patience=30, verbose=0,
                                    restore_best_weights=True)

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

epochs = 6
tsteps = int(len(trainRows)/bs)
vsteps = int(len(valRows)/bs)

history = model.fit_generator(gT(), epochs = epochs, verbose=1, 
                              validation_data = gV(),
                              steps_per_epoch = tsteps, validation_steps=vsteps,
                              callbacks=[lr2, EarLY, checkpoint])

# Predict target

In [ ]:
bestModel=model
bestModel.load_weights(filepath)
predi=bestModel.predict_generator(gTe(), steps=323, verbose=1)
submi = pd.read_csv("../input/jpeg-melanoma-256x256/sample_submission.csv")
submi["target"]=predi
submi.to_csv("submi2CNNs.csv", index=False)

# Distribution of predicted target

In [ ]:
MalignCases=len(np.argwhere(predi > 0.5))

fig, ax = plt.subplots(figsize=(16,6))
ax.set_yscale('log')
sns.distplot(predi, hist_kws={
                 'rwidth': 0.75,
                 'edgecolor': 'black',
                 'alpha': 0.3
             }, color='r', kde=False)
ax.set_title('Final Predictions with ' + str(MalignCases) + ' malign cases')
plt.ylabel("Counts (log scale)")
plt.xlabel("Probability target = 1")
plt.show()

# Plot history

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'],'b')  
plt.plot(history.history['val_loss'],'r')  
plt.xticks(np.arange(0, epochs, epochs/10)) 
plt.xlabel("Num of Epochs")  
plt.ylabel("Loss")  
plt.title("Training Loss vs Validation Loss")  
plt.legend(['train','validation'])
axes = plt.gca()
axes.set_ylim([0,20])

plt.subplot(1, 2, 2)
plt.plot(history.history['auc'],'b')  
plt.plot(history.history['val_auc'],'r')  
plt.xticks(np.arange(0, epochs, epochs/10)) 
plt.xlabel("Num of Epochs")  
plt.ylabel("AUC")  
plt.title("Training AUC vs Validation AUC")  
plt.legend(['train','validation'])
axes = plt.gca()
axes.set_ylim([0,1])

plt.show() 